In [ ]:
#Librerias a utilizar
import socket
import time

SERVER="127.0.0.1" #ip de localhost
PORT=8081 #Puerto a utilizar
cliente=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
cliente.connect((SERVER, PORT)) #Conexion al servidor utilizando ip y puerto
datos_entrada=cliente.recv(1024) #recepcion de datos del servidor en 1024 caracteres
mensaje=datos_entrada.decode("UTF-8")
print(mensaje)
split=mensaje.split()
SERVER=split[0]
s=split[1]
print(s)
if split[0] != '0.0.0.0':
    SERVER=split[0]
    PORT=int(split[1])
    print("Conectando con el servidor: ",SERVER," por el puerto ",PORT)
    cliente.close()
    cliente=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    cliente.connect((SERVER, PORT)) #Conexion al servidor utilizando ip y puerto
    datos_entrada=cliente.recv(1024) #recepcion de datos del servidor en 1024 caracteres
    mensaje=datos_entrada.decode("UTF-8")
else:
    cliente.close()
    raise SystemExit("No se ha encontrado un servidor activo, intentelo mas tarde")

while True:
    #Mensajes sobre el tipo de mensajes para el usuario
    print("¡",mensaje,"!")
    print("Seleccione una opcion\n",
         "1.- Sincronizacion por Christian\n",
         "2.- Sincronizacion por Barkley\n",
         "3.- Salir del servidor\n",
         "4.- Transaccion\n")
    datos_salida = input(">:") #ingreso de datos enviado al servidor
    cliente.sendall(bytes(datos_salida,'UTF-8')) #envio de datos al servidor
    #Sincronizacion por Christian
    if datos_salida=='1':
        #tiempo_ida=time.clock() En desuso en la versión 3.8 de python
        tiempo_ida = time.process_time()
        reloj=cliente.recv(1024) #obtencion de tiempo de reloj
        print("Tiempo de ida ",tiempo_ida)
        print(time.ctime(float(reloj.decode('UTF-8'))+tiempo_ida))
        print("\n")
    #Sincronizacion por Berkley 
    elif datos_salida=='2':
        reloj_actual=time.time()
        cliente.sendall(bytes(str(reloj_actual),'UTF-8'))
        reloj=cliente.recv(1024) #obtencion de tiempo de reloj
        print("Reloj actualizado ",time.ctime(float(reloj.decode('UTF-8'))))
    #Opcion para salir del servidor
    elif datos_salida=='3':
        break
    #Opcion de transaccion
    elif datos_salida=='4':
        print('Ingrese su nombre de usuario')
        datos_salida = input(">:") #ingreso de datos enviado al servidor
        cliente.sendall(bytes(datos_salida,'UTF-8')) #envio de datos al servidor
        res=cliente.recv(1024)
        respuesta=int(res.decode('UTF-8')) #Decodifica el mensaje y lo pasa a entero
        #En caso de encontrar un usuario procede a realizar la transaccion
        if respuesta==1:
            print("Usuario encontrado\n",
            "1.- Depositar\n",
            "2.- Retirar\n",
            "3.- Ver saldo\n")
            datos_salida = input(">:") #ingreso de datos 
            #Deposito
            if datos_salida=='1':
                print("Ingrese la cantidad a depositar\n")
                cliente.sendall(bytes('1','UTF-8')) #envio de respuesta al servidor para que habilite depositos
                datos_salida = input(">:") #ingreso de datos 
                cliente.sendall(bytes(datos_salida,'UTF-8')) #envio de datos al servidor
                res=cliente.recv(1024)
                print("Deposito realizado con exito\n")
            #Retiro
            elif datos_salida=='2':
                print("Ingrese la cantidad a retirar\n")
                cliente.sendall(bytes('2','UTF-8')) #envio de respuesta al servidor para que habilite retiros
                datos_salida = input(">:") #ingreso de datos 
                cliente.sendall(bytes(datos_salida,'UTF-8')) #envio de datos al servidor
                res=cliente.recv(1024)
                print("Retiro realizado con exito\n")
            #Consulta de saldo
            elif datos_salida=='3':
                cliente.sendall(bytes('3','UTF-8')) #envio de respuesta al servidor para consulta de saldo
                res=cliente.recv(1024)
                saldo=int(res.decode('UTF-8')) #Decodifica el mensaje y lo pasa a entero
                print("Su saldo actual es: ",saldo)
            #Otro
            else:
                cliente.sendall(bytes(datos_salida,'UTF-8')) #envio de respuesta al servidor
                print("Error, vuelva a intentarlo mas tarde\n")
        else:
            print("Error, ese usuario no existe\n")
cliente.close()
